In [1]:
from rdkit.Chem.inchi import InchiToInchiKey,MolToInchiKey
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp
pre_spectrums = pp.PassatuttoParser(r'C:\Users\Gosia\Downloads\MassbankOrbi').parse_folder()

processed 100 files
processed 200 files
processed 300 files
processed 400 files
Finished parsing of 457 spectra 


In [2]:
from matchms.importing.load_from_json import as_spectrum
spectrums = []
for i, s in enumerate( pre_spectrums ):
    spectrums.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'%i)

processed 100
processed 200
processed 300
processed 400


In [3]:
from pprint import pprint
name_inchi = {}
for s in spectrums:
    name_inchi[s.get("compound_name")] = s.get("inchi")

In [4]:
random_q_vals = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_EBA.txt', 'r').readlines()  

def inchisEqual(i1, i2):
    return i1.split("/")[:4] == i2.split("/")[:4]
    #return InchiToInchiKey(i1).split('-')[0] == InchiToInchiKey(i2).split('-')[0]
    
from collections import namedtuple
Hit = namedtuple('Hit', ['query', 'score', 'false_hit', 'inchi'])
    
hits = []
name_q_val = {}
for line in random_q_vals[1:]:
    q,t,ti,s,q_val = line.split("\t")
    false_hit = not inchisEqual( name_inchi[q], ti)
    print("inchi query:\n %s \n inchi target:\n %s \n rdkit inchi target:\n %s \n rdkit inchi query:\n %s" % (name_inchi[q],ti,InchiToInchiKey(name_inchi[q]),InchiToInchiKey(ti)))
    #hit = name_inchi[q] != ti
    name_q_val.setdefault(q, []).append( (q_val,ti) )
    hits.append(Hit(q,s,false_hit,ti))

inchi query:
 InChI=1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3 
 inchi target:
 InChI=1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3 
 rdkit inchi target:
 SNPPWIUOZRMYNY-UHFFFAOYSA-N 
 rdkit inchi query:
 SNPPWIUOZRMYNY-UHFFFAOYSA-N
inchi query:
 InChI=1S/C16H14O3/c1-11(16(18)19)13-8-5-9-14(10-13)15(17)12-6-3-2-4-7-12/h2-11H,1H3,(H,18,19) 
 inchi target:
 InChI=1S/C16H14O3/c1-11(16(18)19)13-8-5-9-14(10-13)15(17)12-6-3-2-4-7-12/h2-11H,1H3,(H,18,19) 
 rdkit inchi target:
 DKYWVDODHFEZIM-UHFFFAOYSA-N 
 rdkit inchi query:
 DKYWVDODHFEZIM-UHFFFAOYSA-N
inchi query:
 InChI=1S/C12H20N2O3S/c1-9(2)13-8-12(15)10-4-6-11(7-5-10)14-18(3,16)17/h4-7,9,12-15H,8H2,1-3H3 
 inchi target:
 InChI=1S/C12H20N2O3S/c1-9(2)13-8-12(15)10-4-6-11(7-5-10)14-18(3,16)17/h4-7,9,12-15H,8H2,1-3H3 
 rdkit inchi target:
 ZBMZVLHSJCTVON-UHFFFAOYSA-N 
 rdkit inchi query:
 ZBMZVLHSJCTVON-UHFFFAOYSA-N
inchi query:
 InChI=1S/C14H10F3NO2/c15-14(16,17)9-4-3-5-10(8-9)18-12-7-2

In [5]:
from collections import Counter
c = Counter( h.query for h in hits )
pprint( c.most_common(3) )
print(len(hits))

[('Erythromycin', 2), ('Reserpine', 2), ('Adenosine', 2)]
163


In [7]:
QVal = namedtuple( 'QVal', [ 'query', 'qval', 'inchi', 'false_hit'])

def calculate_q_value(hits):
    hits.sort(key=lambda x: x.score, reverse=True)
    fdr_vals = []
    for i in range(len(hits)):
        #calculate the proportion of false hits so far
        fdr = sum([int(h.false_hit) for h in hits[:i+1]])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    # ensure non-decreasing q_value
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,h in enumerate(hits):
        # Simon says: if not h.false_hit:
        q_list.append(QVal(h.query, q_vals[i], h.inchi, h.false_hit))
    return fdr_vals, q_vals, q_list

In [8]:
import numpy as np
_,_,q_values = calculate_q_value(hits)
for qval in q_values:
    print(qval.qval, qval.false_hit)

0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.01639344262295082 True
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639344262295082 False
0.01639

In [9]:
def take_smaller(x):
    if len(x) == 1:
        return float(x[0][0])
    smaller = 0
    for i, (n,_) in enumerate(x):
        if float(n) < float(x[smaller][0]):
            smaller = i
    return float(x.pop(smaller)[0])

count = 0
print('Passatutto,   Ours,      Are They the same?     Name')
for name,q_val,inchi,_ in q_values:
    print('%4f      %4f    %s            %s' %( take_smaller(name_q_val[name]), float(q_val), inchisEqual(name_inchi[name], inchi),name))
    

Passatutto,   Ours,      Are They the same?     Name
0.003876      0.000000    True            Bupropion
0.003876      0.000000    True            Ketoprofen
0.003876      0.000000    True            Sotalol
0.003876      0.000000    True            Flufenamic acid
0.003876      0.000000    True            Moclobemide
0.003876      0.000000    True            Mefenamic acid
0.003876      0.000000    True            Gabapentin
0.003876      0.000000    True            Venlafaxine
0.003876      0.000000    True            Clarithromycin
0.003876      0.000000    True            Metoclopramide
0.003876      0.000000    True            Fluconazole
0.003876      0.000000    True            Terbutryn
0.003876      0.000000    True            Clozapine
0.003876      0.000000    True            1-(3-(Trifluoromethyl)phenyl)piperazine
0.003876      0.000000    True            Sulfathiazole
0.003876      0.000000    True            Amisulpride
0.003876      0.000000    True            Sulfapyrid

In [10]:
import pylab as plt
plt.figure(figsize=(8,6))
plt.plot(q_value_list_b, q_value_list, label='random')
plt.plot([0,0.4], [0,0.4], 'k--')
plt.xlabel('True')
plt.ylabel('Estimated')
plt.legend()
plt.grid()

NameError: name 'q_value_list_b' is not defined

<Figure size 576x432 with 0 Axes>

# Aminopyrine example:
Query:  Aminopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3
Target: Amidopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3
    
# Sebuthylazine example
Query: Sebuthylazine 
Target: MLS001048955-01!2,2-dimethyl-4,10-dihydro-3H-pyrano[2,3-b]quinolin-5-one
InChI=1S/C9H16ClN5/c1-4-6(3)12-9-14-7(10)13-8(15-9)11-5-2/h6H,4-5H2,1-3H3,(H2,11,12,13,14,15)
InChI=1S/C14H15NO2/c1-14(2)8-7-10-12(16)9-5-3-4-6-11(9)15-13(10)17-14/h3-6H,7-8H2,1-2H3,(H,15,16)